# Mandelbrot: numpy vs. numba vectorization
----

## Requirements

In [ ]:
!conda install -c conda-forge numba -y

## Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numba

**Define the escape time function**

In [ ]:
def escape_time(p, maxtime):
    """Perform the Mantelbrot iteration until it's clear that $p$ diverges 
    or the maximum number of iterations has been reached.
    
    Parameters
    ----------
    p: complex
        point in the complex plane
    maxtime: int
        maximum number of iterations to perform
    """
    z = 0j
    for i in range(maxtime):
        z = z ** 2 + p
        if abs(z) > 2:
            return i
    return maxtime

**Setup**

In [ ]:
# Problem size
n = 2048
maxtime = 100

# Domain
xmin, xmax = -2.2, 1.5
ymin, ymax = -1.5, 1.5

DTYPE_COMPLEX = 'c8'
DTYPE_INT = 'i4'

x = np.linspace(xmin + 0j, xmax + 0j, n,
                dtype = np.dtype(DTYPE_COMPLEX))
y = np.linspace(ymin*1j, ymax*1j, n,
                dtype = np.dtype(DTYPE_COMPLEX))
x, y = np.meshgrid(x, y)

# Define complex plane
p = x + y

In [ ]:
del x, y

In [ ]:
%whos

## Numpy vectorize

In [ ]:
escape_time_numpy = np.vectorize(escape_time)

In [ ]:
%%time 
t = escape_time_numpy(p, maxtime)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(t, cmap='viridis')

## Numba vectorize (+ multithreaded)

In [ ]:
escape_time_numba = numba.vectorize(['{0}({1},{0})'.format(DTYPE_INT,
                                                           DTYPE_COMPLEX)],
                                    nopython=True,
                                    target='parallel')(escape_time)

In [ ]:
%%time
t = escape_time_numba(p, maxtime)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.gca()
ax.imshow(t, origin='lower', extent=(xmin,xmax,ymin,ymax),
         cmap='viridis_r')
ax.axis('scaled')